## EDA documento .JSON

Luis Angel Garcia (2230177)

## Librerias

In [1]:
import pandas as pd
import logging
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import jsonlines
import pandas as pd
import ast 
from IPython.display import display
from sklearn.preprocessing import StandardScaler, LabelEncoder

sys.path.append("../config_documents")

## Configuracion del login

In [2]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

## Cargar archivo

In [3]:
data = []
with jsonlines.open("../source/MLA_100k.jsonlines") as reader:
    for obj in reader:
        data.append(obj)
df = pd.DataFrame(data)

df.to_csv("../source/productos.csv", index=False)


In [4]:
# Cargar el CSV original
df = pd.read_csv("../source/productos.csv")

# Lista de columnas que contienen estructuras anidadas como strings
columnas_anidadas = [
    'seller_address', 'shipping', 'geolocation',
    'non_mercado_pago_payment_methods', 'pictures',
    'attributes', 'descriptions'
]

# Convertir strings a estructuras Python (dicts o lists)
for col in columnas_anidadas:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)

# ======= Extracción de campos útiles =======

# seller_address
df['latitude'] = df['seller_address'].apply(lambda x: x.get('latitude') if x else None)
df['longitude'] = df['seller_address'].apply(lambda x: x.get('longitude') if x else None)
df['country'] = df['seller_address'].apply(lambda x: x.get('country', {}).get('name') if x else None)
df['state'] = df['seller_address'].apply(lambda x: x.get('state', {}).get('name') if x else None)
df['city'] = df['seller_address'].apply(lambda x: x.get('city', {}).get('name') if x else None)
df['neighborhood'] = df['seller_address'].apply(lambda x: x.get('search_location', {}).get('neighborhood', {}).get('name') if x else None)

# shipping
df['free_shipping'] = df['shipping'].apply(lambda x: x.get('free_shipping') if x else None)
df['shipping_mode'] = df['shipping'].apply(lambda x: x.get('mode') if x else None)
df['local_pick_up'] = df['shipping'].apply(lambda x: x.get('local_pick_up') if x else None)

# geolocation (si no es redundante con lat/lon anterior)
df['geo_lat'] = df['geolocation'].apply(lambda x: x.get('latitude') if x else None)
df['geo_lon'] = df['geolocation'].apply(lambda x: x.get('longitude') if x else None)

# pictures: contar cuántas tiene
df['num_pictures'] = df['pictures'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# métodos de pago no-mercado-pago
df['num_non_mp_methods'] = df['non_mercado_pago_payment_methods'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# título: longitud
df['title_length'] = df['title'].apply(lambda x: len(x) if isinstance(x, str) else 0)

# diferencia entre precio y precio original
df['price_diff'] = df['price'] - df['base_price']

columnas_finales = [
    'id', 'title', 'condition', 'price', 'base_price', 'price_diff',
    'sold_quantity', 'available_quantity', 'accepts_mercadopago',
    'free_shipping', 'shipping_mode', 'local_pick_up',
    'latitude', 'longitude', 'country', 'state', 'city', 'neighborhood',
    'num_pictures', 'num_non_mp_methods', 'title_length'
]

df_final = df[columnas_finales].copy()

# Guardar el resultado
df_final.to_csv("../source/products_dataset.csv", index=False)

## Analisis basico

In [5]:
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.width', None)  # Ajusta el ancho de la salida
pd.set_option('display.max_rows', None)

display('Primeras filas del dataset:\n',df_final.head())


'Primeras filas del dataset:\n'

,id,title,condition,price,base_price,price_diff,sold_quantity,available_quantity,accepts_mercadopago,free_shipping,shipping_mode,local_pick_up,latitude,longitude,country,state,city,neighborhood,num_pictures,num_non_mp_methods,title_length
0,MLA578052519,Auriculares Samsung Originales Manos Libres Ca...,new,80.0,80.0,0.0,0,1,True,False,not_specified,True,-34.62807,-58.398671,Argentina,Capital Federal,San Cristóbal,San Cristóbal,2,3,60
1,MLA581565358,Cuchillo Daga Acero Carbón Casco Yelmo Solinge...,used,2650.0,2650.0,0.0,0,1,True,False,me2,True,-34.593552,-58.505917,Argentina,Capital Federal,Buenos Aires,Villa Devoto,6,2,57
2,MLA578780872,"Antigua Revista Billiken, N° 1826, Año 1954",used,60.0,60.0,0.0,0,1,True,False,me2,True,-34.623391,-58.414395,Argentina,Capital Federal,Boedo,Boedo,1,3,43
3,MLA581877385,Alarma Guardtex Gx412 Seguridad Para El Automo...,new,580.0,580.0,0.0,0,1,True,False,me2,True,-34.628189,-58.492921,Argentina,Capital Federal,Floresta,Floresta,2,3,52
4,MLA576112692,Serenata - Jennifer Blake,used,30.0,30.0,0.0,0,1,True,False,not_specified,True,-34.634655,-58.549504,Argentina,Buenos Aires,Tres de febrero,,2,3,25


## Graficas

In [6]:
def plot_target_distribution(df_cleaned):
    sns.countplot(x='condition', data=df_cleaned)
    plt.title('Distribución de la variable objetivo: Condition')
    plt.show()

## Limpieza y transformacion

In [7]:
# 1. Eliminar columnas irrelevantes
def clean_columns(df_final):
    columns_to_remove = ['id', 'title', 'local_pick_up', 'latitude', 'longitude']  # Añadimos 'latitude' y 'longitude' a la lista
    df_cleaned = df_final.drop(columns=columns_to_remove, errors='ignore')  # errors='ignore' para no generar error si alguna columna no existe
    return df_cleaned

# 2. Codificar la variable objetivo 'condition' (new -> 1, used -> 0)
def encode_condition(df_cleaned):
    # Asegurarse de que los valores de 'condition' sean cadenas de texto
    if df_cleaned['condition'].dtype != 'object':  # Si no es texto
        # Si la columna 'condition' tiene valores numéricos (1.0 y 0.0), mapeamos a 'new' y 'used'
        df_cleaned['condition'] = df_cleaned['condition'].map({1.0: 'new', 0.0: 'used'})
    
    # Codificar 'condition' como numérico: 'new' -> 1, 'used' -> 0
    le = LabelEncoder()
    df_cleaned['condition'] = le.fit_transform(df_cleaned['condition'])
    
    # Forzar la conversión de la columna 'condition' a enteros (para eliminar flotantes)
    df_cleaned['condition'] = df_cleaned['condition'].astype('int')
    
    # Verificar los valores únicos en 'condition' después de la codificación
    print(f"Valores únicos en 'condition' después de la codificación: {df_cleaned['condition'].unique()}")
    print(f"Tipo de 'condition' después de la conversión: {df_cleaned['condition'].dtype}")
    
    return df_cleaned

# 3. Codificar columnas categóricas (como 'shipping_mode', 'accepts_mercadopago')
def encode_categorical_columns(df_cleaned):
    # Codificar columnas con valores de tipo texto a numérico
    categorical_columns = ['shipping_mode', 'accepts_mercadopago', 'free_shipping', 'country', 'state', 'city', 'neighborhood']
    
    for col in categorical_columns:
        df_cleaned[col] = df_cleaned[col].astype('category').cat.codes  # Codificar los valores categóricos en números

    return df_cleaned

# 4. Escalar las características numéricas
def scale_features(df_cleaned, numerical_columns):
    scaler = StandardScaler()
    df_cleaned[numerical_columns] = scaler.fit_transform(df_cleaned[numerical_columns])
    return df_cleaned

# 5. Función principal para realizar el preprocesamiento
# 5. Función principal para realizar el preprocesamiento
def preprocess_data(df_final):
    # Limpiar columnas irrelevantes
    df_cleaned = clean_columns(df_final)
    
    # Codificar la variable objetivo 'condition'
    df_cleaned = encode_condition(df_cleaned)
    
    # Codificar columnas categóricas
    df_cleaned = encode_categorical_columns(df_cleaned)
    
    # Seleccionar solo las columnas numéricas para trabajar con ellas
    numerical_columns = df_cleaned.select_dtypes(include=np.number).columns

    # Escalar las características numéricas
    df_cleaned = scale_features(df_cleaned, numerical_columns)
    
    # Asegurarnos de que 'condition' esté como entero antes de guardar el CSV
    df_cleaned['condition'] = df_cleaned['condition'].astype(int)

    # Separar las características (X) de la variable objetivo (y)
    X = df_cleaned.drop(columns=['condition'])  # Características
    y = df_cleaned['condition']  # Variable objetivo
    
    # Guardar el DataFrame limpio en un archivo CSV
    df_cleaned.to_csv('../source/dataset_limpio.csv', index=False)

    # Verificar el tipo de la columna 'condition' después de la conversión y guardar
    print(f"Tipo de 'condition' después de guardar como CSV: {df_cleaned['condition'].dtype}")
    
    return X, y


# Ejecutar el preprocesamiento
X, y = preprocess_data(df_final)




Valores únicos en 'condition' después de la codificación: [0 1]
Tipo de 'condition' después de la conversión: int64
Tipo de 'condition' después de guardar como CSV: int64
